In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time

In [3]:
all_teams = [] ## list to store all teams

html = requests.get('https://fbref.com/en/comps/9/Premier-League-Stats').text ##getting the html from the website
soup = BeautifulSoup(html, 'lxml')
table = soup.find_all('table', class_ = 'stats_table')[0] ##only want the first table, therefore the first index

links = table.find_all('a') ## finding all links in the table 
links = [l.get("href") for l in links] ##parsing through links
links = [l for l in links if '/squads/' in l] ##filtering through links to only get squads

team_urls = [f"https://fåbref.com{l}" for l in links] ## formatting back to links

for team_url in team_urls: 
    team_name = team_url.split("/")[-1].replace("-Stats", "") ##isolating the names of the teams
    data = requests.get(team_url).text
    soup = BeautifulSoup(data, 'lxml')
    stats = soup.find_all('table', class_ = "stats_table")[0] ##again, only want the first table

    if stats and stats.columns: stats.columns = stats.columns.droplevel() ##formatting the stats

    # Assuming 'team_data' is a BeautifulSoup Tag
    # Convert it into a DataFrame
    team_data = pd.read_html(str(stats))[0]
    team_data["Team"]= team_name
    all_teams.append(team_data) ## appending the data
    time.sleep(5) ## making sure we don't get blocked from scraping by delaying each loop by 5 seconds

stat_df = pd.concat(all_teams) ## concatenating all of the stats

In [109]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time

all_teams = []

base_url = 'https://fbref.com'

# Get Premier League main page
html = requests.get('https://fbref.com/en/comps/9/Premier-League-Stats').text
soup = BeautifulSoup(html, 'lxml')

# Get first table containing teams
table = soup.find_all('table', class_='stats_table')[0]

# Parse rows to get team id and name together
rows = table.find_all('tr')
for row in rows:
    a_tag = row.find('a')
    if not a_tag:
        continue
    href = a_tag.get('href')
    if href and '/squads/' in href:
        parts = href.split('/')
        if len(parts) < 5:
            continue
        team_id = parts[3]
        team_name_raw = parts[-1]
        team_name = team_name_raw.replace('-Stats', '')
        
        # Construct team URL for 2024–2025 season
        team_url = f"{base_url}/en/squads/{team_id}/2024-2025/{team_name}-Stats"
        print(f"Scraping {team_url}")
        
        # Get team page
        data = requests.get(team_url).text
        soup_team = BeautifulSoup(data, 'lxml')

        # Get all stats tables on the page
        stats_tables = soup_team.find_all('table', class_='stats_table')
        if not stats_tables:
            print(f"No stats table found for {team_url}")
            continue

        stats_table = stats_tables[0]

        # Remove <tfoot> section (which contains "Squad Total", "Opponent Total")
        tfoot = stats_table.find('tfoot')
        if tfoot:
            tfoot.decompose()  # This removes it from the BeautifulSoup tree

        # Parse cleaned table using pandas
        filtered_table_html = str(stats_table)
        team_data = pd.read_html(filtered_table_html)[0]
        team_data['Team'] = team_name
        all_teams.append(team_data)

        time.sleep(5)  # Be nice to the server

# Combine all team data into one DataFrame
stat_df = pd.concat(all_teams, ignore_index=True)


Scraping https://fbref.com/en/squads/18bb7c10/2024-2025/Arsenal-Stats
Scraping https://fbref.com/en/squads/8602292d/2024-2025/Aston-Villa-Stats
Scraping https://fbref.com/en/squads/4ba7cbea/2024-2025/Bournemouth-Stats
Scraping https://fbref.com/en/squads/cd051869/2024-2025/Brentford-Stats
Scraping https://fbref.com/en/squads/d07537b9/2024-2025/Brighton-and-Hove-Albion-Stats
Scraping https://fbref.com/en/squads/943e8050/2024-2025/Burnley-Stats
Scraping https://fbref.com/en/squads/cff3d9bb/2024-2025/Chelsea-Stats
Scraping https://fbref.com/en/squads/47c64c55/2024-2025/Crystal-Palace-Stats
Scraping https://fbref.com/en/squads/d3fd31cc/2024-2025/Everton-Stats
Scraping https://fbref.com/en/squads/fd962109/2024-2025/Fulham-Stats
Scraping https://fbref.com/en/squads/5bfb9659/2024-2025/Leeds-United-Stats
Scraping https://fbref.com/en/squads/822bd0ba/2024-2025/Liverpool-Stats
Scraping https://fbref.com/en/squads/b8fd03ef/2024-2025/Manchester-City-Stats
Scraping https://fbref.com/en/squads/19538

In [110]:
print(stat_df.isna().sum())

Unnamed: 0_level_0   Player        0
Unnamed: 1_level_0   Nation        1
Unnamed: 2_level_0   Pos           0
Unnamed: 3_level_0   Age           1
Unnamed: 4_level_0   MP           63
Playing Time         Starts        0
                     Min         137
                     90s         137
Performance          Gls         137
                     Ast         137
                     G+A         137
                     G-PK        137
                     PK          137
                     PKatt       137
                     CrdY        137
                     CrdR        137
Expected             xG          137
                     npxG        137
                     xAG         137
                     npxG+xAG    137
Progression          PrgC        137
                     PrgP        137
                     PrgR        137
Per 90 Minutes       Gls         137
                     Ast         137
                     G+A         137
                     G-PK        137
 

In [111]:
stat_df

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
                Player             Nation                Pos   
0           David Raya             es ESP                 GK   
1       William Saliba             fr FRA                 DF   
2          Declan Rice            eng ENG                 MF   
3        Thomas Partey             gh GHA              MF,DF   
4     Leandro Trossard             be BEL                 FW   
..                 ...                ...                ...   
702    Tawanda Chirewa             zw ZIM                 MF   
703        Luke Cundle            eng ENG                 MF   
704         Tom Edozie            eng ENG                 MF   
705   Bastien Meupiyou             fr FRA                 DF   
706     Wesley Okoduwa            eng ENG                 FW   

    Unnamed: 3_level_0 Unnamed: 4_level_0 Playing Time                \
                   Age                 MP       Starts     Min   90s   
0                 28.0               38.0           38  3420.0  38.0   
1                 23.0               35.0           35  3039.0  33.8   
2                 25.0               35.0           33  2825.0  31.4   
3                 31.0               35.0           31  2797.0  31.1   
4                 29.0               38.0           28  2546.0  28.3   
..                 ...                ...          ...     ...   ...   
702               20.0                0.0            0     NaN   NaN   
703               22.0                0.0            0     NaN   NaN   
704               18.0                0.0            0     NaN   NaN   
705               18.0                0.0            0     NaN   NaN   
706               16.0                0.0            0     NaN   NaN   

    Performance       ... Per 90 Minutes                                  \
            Gls  Ast  ...           G-PK G+A-PK    xG   xAG xG+xAG  npxG   
0           0.0  0.0  ...           0.00   0.00  0.00  0.00   0.00  0.00   
1           2.0  0.0  ...           0.06   0.06  0.07  0.03   0.09  0.07   
2           4.0  7.0  ...           0.13   0.35  0.11  0.21   0.32  0.11   
3           4.0  2.0  ...           0.13   0.19  0.07  0.07   0.14  0.07   
4           8.0  7.0  ...           0.28   0.53  0.25  0.22   0.47  0.25   
..          ...  ...  ...            ...    ...   ...   ...    ...   ...   
702         NaN  NaN  ...            NaN    NaN   NaN   NaN    NaN   NaN   
703         NaN  NaN  ...            NaN    NaN   NaN   NaN    NaN   NaN   
704         NaN  NaN  ...            NaN    NaN   NaN   NaN    NaN   NaN   
705         NaN  NaN  ...            NaN    NaN   NaN   NaN    NaN   NaN   
706         NaN  NaN  ...            NaN    NaN   NaN   NaN    NaN   NaN   

             Unnamed: 33_level_0                     Team Playing Time  
    npxG+xAG             Matches                                    MP  
0       0.00             Matches                  Arsenal          NaN  
1       0.09             Matches                  Arsenal          NaN  
2       0.32             Matches                  Arsenal          NaN  
3       0.14             Matches                  Arsenal          NaN  
4       0.47             Matches                  Arsenal          NaN  
..       ...                 ...                      ...          ...  
702      NaN             Matches  Wolverhampton-Wanderers          NaN  
703      NaN             Matches  Wolverhampton-Wanderers          NaN  
704      NaN             Matches  Wolverhampton-Wanderers          NaN  
705      NaN             Matches  Wolverhampton-Wanderers          NaN  
706      NaN             Matches  Wolverhampton-Wanderers          NaN  

[707 rows x 36 columns]

In [112]:
stat_df.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in stat_df.columns]

In [113]:
stat_df.drop(columns=["Playing Time_MP"], inplace=True)

In [114]:
stat_df.rename(columns={
    'Unnamed: 0_level_0_Player': 'Player Name',
    'Unnamed: 1_level_0_Nation': 'Nation',
    'Unnamed: 2_level_0_Pos': 'Position',
    'Unnamed: 3_level_0_Age': 'Age',
    'Unnamed: 4_level_0_MP': 'Matches Played'
}, inplace=True)

In [115]:
stat_df

,Player Name,Nation,Position,Age,Matches Played,Playing Time_Starts,Playing Time_Min,Playing Time_90s,Performance_Gls,Performance_Ast,...,Per 90 Minutes_G+A,Per 90 Minutes_G-PK,Per 90 Minutes_G+A-PK,Per 90 Minutes_xG,Per 90 Minutes_xAG,Per 90 Minutes_xG+xAG,Per 90 Minutes_npxG,Per 90 Minutes_npxG+xAG,Unnamed: 33_level_0_Matches,Team_
0,David Raya,es ESP,GK,28.0,38.0,38,3420.0,38.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches,Arsenal
1,William Saliba,fr FRA,DF,23.0,35.0,35,3039.0,33.8,2.0,0.0,...,0.06,0.06,0.06,0.07,0.03,0.09,0.07,0.09,Matches,Arsenal
2,Declan Rice,eng ENG,MF,25.0,35.0,33,2825.0,31.4,4.0,7.0,...,0.35,0.13,0.35,0.11,0.21,0.32,0.11,0.32,Matches,Arsenal
3,Thomas Partey,gh GHA,"MF,DF",31.0,35.0,31,2797.0,31.1,4.0,2.0,...,0.19,0.13,0.19,0.07,0.07,0.14,0.07,0.14,Matches,Arsenal
4,Leandro Trossard,be BEL,FW,29.0,38.0,28,2546.0,28.3,8.0,7.0,...,0.53,0.28,0.53,0.25,0.22,0.47,0.25,0.47,Matches,Arsenal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702,Tawanda Chirewa,zw ZIM,MF,20.0,0.0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matches,Wolverhampton-Wanderers
703,Luke Cundle,eng ENG,MF,22.0,0.0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matches,Wolverhampton-Wanderers
704,Tom Edozie,eng ENG,MF,18.0,0.0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matches,Wolverhampton-Wanderers
705,Bastien Meupiyou,fr FRA,DF,18.0,0.0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matches,Wolverhampton-Wanderers


In [118]:
print(stat_df.isna().sum())

Player Name                    0
Nation                         0
Position                       0
Age                            0
Matches Played                 0
Playing Time_Starts            0
Playing Time_Min               0
Playing Time_90s               0
Performance_Gls                0
Performance_Ast                0
Performance_G+A                0
Performance_G-PK               0
Performance_PK                 0
Performance_PKatt              0
Performance_CrdY               0
Performance_CrdR               0
Expected_xG                    0
Expected_npxG                  0
Expected_xAG                   0
Expected_npxG+xAG              0
Progression_PrgC               0
Progression_PrgP               0
Progression_PrgR               0
Per 90 Minutes_Gls             0
Per 90 Minutes_Ast             0
Per 90 Minutes_G+A             0
Per 90 Minutes_G-PK            0
Per 90 Minutes_G+A-PK          0
Per 90 Minutes_xG              0
Per 90 Minutes_xAG             0
Per 90 Min

In [116]:
stat_df.dropna(inplace=True)

In [117]:
stat_df

,Player Name,Nation,Position,Age,Matches Played,Playing Time_Starts,Playing Time_Min,Playing Time_90s,Performance_Gls,Performance_Ast,...,Per 90 Minutes_G+A,Per 90 Minutes_G-PK,Per 90 Minutes_G+A-PK,Per 90 Minutes_xG,Per 90 Minutes_xAG,Per 90 Minutes_xG+xAG,Per 90 Minutes_npxG,Per 90 Minutes_npxG+xAG,Unnamed: 33_level_0_Matches,Team_
0,David Raya,es ESP,GK,28.0,38.0,38,3420.0,38.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches,Arsenal
1,William Saliba,fr FRA,DF,23.0,35.0,35,3039.0,33.8,2.0,0.0,...,0.06,0.06,0.06,0.07,0.03,0.09,0.07,0.09,Matches,Arsenal
2,Declan Rice,eng ENG,MF,25.0,35.0,33,2825.0,31.4,4.0,7.0,...,0.35,0.13,0.35,0.11,0.21,0.32,0.11,0.32,Matches,Arsenal
3,Thomas Partey,gh GHA,"MF,DF",31.0,35.0,31,2797.0,31.1,4.0,2.0,...,0.19,0.13,0.19,0.07,0.07,0.14,0.07,0.14,Matches,Arsenal
4,Leandro Trossard,be BEL,FW,29.0,38.0,28,2546.0,28.3,8.0,7.0,...,0.53,0.28,0.53,0.25,0.22,0.47,0.25,0.47,Matches,Arsenal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,Daniel Podence,pt POR,"MF,FW",28.0,2.0,0,48.0,0.5,0.0,0.0,...,0.00,0.00,0.00,0.07,0.00,0.07,0.07,0.07,Matches,Wolverhampton-Wanderers
697,Chiquinho,pt POR,MF,24.0,1.0,0,7.0,0.1,0.0,0.0,...,0.00,0.00,0.00,0.00,0.42,0.42,0.00,0.42,Matches,Wolverhampton-Wanderers
699,Tom King,wls WAL,GK,29.0,1.0,0,1.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches,Wolverhampton-Wanderers
700,Alfie Pond,eng ENG,DF,20.0,1.0,0,1.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches,Wolverhampton-Wanderers


In [120]:
stat_df[stat_df['Player Name'] == "Jobe Bellingham"]

,Player Name,Nation,Position,Age,Matches Played,Playing Time_Starts,Playing Time_Min,Playing Time_90s,Performance_Gls,Performance_Ast,...,Per 90 Minutes_G+A,Per 90 Minutes_G-PK,Per 90 Minutes_G+A-PK,Per 90 Minutes_xG,Per 90 Minutes_xAG,Per 90 Minutes_xG+xAG,Per 90 Minutes_npxG,Per 90 Minutes_npxG+xAG,Unnamed: 33_level_0_Matches,Team_
565,Jobe Bellingham,eng ENG,MF,18.0,40.0,39,3507.0,39.0,4.0,3.0,...,0.18,0.1,0.18,0.08,0.07,0.15,0.08,0.15,Matches,Sunderland


In [121]:
stat_df.drop(columns=["Unnamed: 33_level_0_Matches"], inplace=True)

In [122]:
stat_df

,Player Name,Nation,Position,Age,Matches Played,Playing Time_Starts,Playing Time_Min,Playing Time_90s,Performance_Gls,Performance_Ast,...,Per 90 Minutes_Ast,Per 90 Minutes_G+A,Per 90 Minutes_G-PK,Per 90 Minutes_G+A-PK,Per 90 Minutes_xG,Per 90 Minutes_xAG,Per 90 Minutes_xG+xAG,Per 90 Minutes_npxG,Per 90 Minutes_npxG+xAG,Team_
0,David Raya,es ESP,GK,28.0,38.0,38,3420.0,38.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Arsenal
1,William Saliba,fr FRA,DF,23.0,35.0,35,3039.0,33.8,2.0,0.0,...,0.00,0.06,0.06,0.06,0.07,0.03,0.09,0.07,0.09,Arsenal
2,Declan Rice,eng ENG,MF,25.0,35.0,33,2825.0,31.4,4.0,7.0,...,0.22,0.35,0.13,0.35,0.11,0.21,0.32,0.11,0.32,Arsenal
3,Thomas Partey,gh GHA,"MF,DF",31.0,35.0,31,2797.0,31.1,4.0,2.0,...,0.06,0.19,0.13,0.19,0.07,0.07,0.14,0.07,0.14,Arsenal
4,Leandro Trossard,be BEL,FW,29.0,38.0,28,2546.0,28.3,8.0,7.0,...,0.25,0.53,0.28,0.53,0.25,0.22,0.47,0.25,0.47,Arsenal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,Daniel Podence,pt POR,"MF,FW",28.0,2.0,0,48.0,0.5,0.0,0.0,...,0.00,0.00,0.00,0.00,0.07,0.00,0.07,0.07,0.07,Wolverhampton-Wanderers
697,Chiquinho,pt POR,MF,24.0,1.0,0,7.0,0.1,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.42,0.42,0.00,0.42,Wolverhampton-Wanderers
699,Tom King,wls WAL,GK,29.0,1.0,0,1.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Wolverhampton-Wanderers
700,Alfie Pond,eng ENG,DF,20.0,1.0,0,1.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Wolverhampton-Wanderers


In [123]:
stat_df['League'] = 'English Premier League'

In [124]:
stat_df

,Player Name,Nation,Position,Age,Matches Played,Playing Time_Starts,Playing Time_Min,Playing Time_90s,Performance_Gls,Performance_Ast,...,Per 90 Minutes_G+A,Per 90 Minutes_G-PK,Per 90 Minutes_G+A-PK,Per 90 Minutes_xG,Per 90 Minutes_xAG,Per 90 Minutes_xG+xAG,Per 90 Minutes_npxG,Per 90 Minutes_npxG+xAG,Team_,League
0,David Raya,es ESP,GK,28.0,38.0,38,3420.0,38.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Arsenal,English Premier League
1,William Saliba,fr FRA,DF,23.0,35.0,35,3039.0,33.8,2.0,0.0,...,0.06,0.06,0.06,0.07,0.03,0.09,0.07,0.09,Arsenal,English Premier League
2,Declan Rice,eng ENG,MF,25.0,35.0,33,2825.0,31.4,4.0,7.0,...,0.35,0.13,0.35,0.11,0.21,0.32,0.11,0.32,Arsenal,English Premier League
3,Thomas Partey,gh GHA,"MF,DF",31.0,35.0,31,2797.0,31.1,4.0,2.0,...,0.19,0.13,0.19,0.07,0.07,0.14,0.07,0.14,Arsenal,English Premier League
4,Leandro Trossard,be BEL,FW,29.0,38.0,28,2546.0,28.3,8.0,7.0,...,0.53,0.28,0.53,0.25,0.22,0.47,0.25,0.47,Arsenal,English Premier League
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,Daniel Podence,pt POR,"MF,FW",28.0,2.0,0,48.0,0.5,0.0,0.0,...,0.00,0.00,0.00,0.07,0.00,0.07,0.07,0.07,Wolverhampton-Wanderers,English Premier League
697,Chiquinho,pt POR,MF,24.0,1.0,0,7.0,0.1,0.0,0.0,...,0.00,0.00,0.00,0.00,0.42,0.42,0.00,0.42,Wolverhampton-Wanderers,English Premier League
699,Tom King,wls WAL,GK,29.0,1.0,0,1.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Wolverhampton-Wanderers,English Premier League
700,Alfie Pond,eng ENG,DF,20.0,1.0,0,1.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Wolverhampton-Wanderers,English Premier League


In [126]:
non_epl_teams = ['Sunderland', 'Burnley', 'Leeds United']
stat_df.loc[stat_df['Team_'].isin(non_epl_teams), 'League'] = 'EFL Championship'

In [127]:
stat_df

,Player Name,Nation,Position,Age,Matches Played,Playing Time_Starts,Playing Time_Min,Playing Time_90s,Performance_Gls,Performance_Ast,...,Per 90 Minutes_G+A,Per 90 Minutes_G-PK,Per 90 Minutes_G+A-PK,Per 90 Minutes_xG,Per 90 Minutes_xAG,Per 90 Minutes_xG+xAG,Per 90 Minutes_npxG,Per 90 Minutes_npxG+xAG,Team_,League
0,David Raya,es ESP,GK,28.0,38.0,38,3420.0,38.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Arsenal,English Premier League
1,William Saliba,fr FRA,DF,23.0,35.0,35,3039.0,33.8,2.0,0.0,...,0.06,0.06,0.06,0.07,0.03,0.09,0.07,0.09,Arsenal,English Premier League
2,Declan Rice,eng ENG,MF,25.0,35.0,33,2825.0,31.4,4.0,7.0,...,0.35,0.13,0.35,0.11,0.21,0.32,0.11,0.32,Arsenal,English Premier League
3,Thomas Partey,gh GHA,"MF,DF",31.0,35.0,31,2797.0,31.1,4.0,2.0,...,0.19,0.13,0.19,0.07,0.07,0.14,0.07,0.14,Arsenal,English Premier League
4,Leandro Trossard,be BEL,FW,29.0,38.0,28,2546.0,28.3,8.0,7.0,...,0.53,0.28,0.53,0.25,0.22,0.47,0.25,0.47,Arsenal,English Premier League
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,Daniel Podence,pt POR,"MF,FW",28.0,2.0,0,48.0,0.5,0.0,0.0,...,0.00,0.00,0.00,0.07,0.00,0.07,0.07,0.07,Wolverhampton-Wanderers,English Premier League
697,Chiquinho,pt POR,MF,24.0,1.0,0,7.0,0.1,0.0,0.0,...,0.00,0.00,0.00,0.00,0.42,0.42,0.00,0.42,Wolverhampton-Wanderers,English Premier League
699,Tom King,wls WAL,GK,29.0,1.0,0,1.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Wolverhampton-Wanderers,English Premier League
700,Alfie Pond,eng ENG,DF,20.0,1.0,0,1.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Wolverhampton-Wanderers,English Premier League


In [128]:
stat_df.rename(columns = {
    'Team_': 'Team'
}, inplace=True)

In [130]:
stat_df

,Player Name,Nation,Position,Age,Matches Played,Playing Time_Starts,Playing Time_Min,Playing Time_90s,Performance_Gls,Performance_Ast,...,Per 90 Minutes_G+A,Per 90 Minutes_G-PK,Per 90 Minutes_G+A-PK,Per 90 Minutes_xG,Per 90 Minutes_xAG,Per 90 Minutes_xG+xAG,Per 90 Minutes_npxG,Per 90 Minutes_npxG+xAG,Team,League
0,David Raya,es ESP,GK,28.0,38.0,38,3420.0,38.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Arsenal,English Premier League
1,William Saliba,fr FRA,DF,23.0,35.0,35,3039.0,33.8,2.0,0.0,...,0.06,0.06,0.06,0.07,0.03,0.09,0.07,0.09,Arsenal,English Premier League
2,Declan Rice,eng ENG,MF,25.0,35.0,33,2825.0,31.4,4.0,7.0,...,0.35,0.13,0.35,0.11,0.21,0.32,0.11,0.32,Arsenal,English Premier League
3,Thomas Partey,gh GHA,"MF,DF",31.0,35.0,31,2797.0,31.1,4.0,2.0,...,0.19,0.13,0.19,0.07,0.07,0.14,0.07,0.14,Arsenal,English Premier League
4,Leandro Trossard,be BEL,FW,29.0,38.0,28,2546.0,28.3,8.0,7.0,...,0.53,0.28,0.53,0.25,0.22,0.47,0.25,0.47,Arsenal,English Premier League
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,Daniel Podence,pt POR,"MF,FW",28.0,2.0,0,48.0,0.5,0.0,0.0,...,0.00,0.00,0.00,0.07,0.00,0.07,0.07,0.07,Wolverhampton-Wanderers,English Premier League
697,Chiquinho,pt POR,MF,24.0,1.0,0,7.0,0.1,0.0,0.0,...,0.00,0.00,0.00,0.00,0.42,0.42,0.00,0.42,Wolverhampton-Wanderers,English Premier League
699,Tom King,wls WAL,GK,29.0,1.0,0,1.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Wolverhampton-Wanderers,English Premier League
700,Alfie Pond,eng ENG,DF,20.0,1.0,0,1.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Wolverhampton-Wanderers,English Premier League


In [145]:
stat_df[stat_df['Team'].isin(["Sunderland", "Leeds-United", "Burnley"])]

,Player Name,Nation,Position,Age,Matches Played,Playing Time_Starts,Playing Time_Min,Playing Time_90s,Performance_Gls,Performance_Ast,...,Per 90 Minutes_G+A,Per 90 Minutes_G-PK,Per 90 Minutes_G+A-PK,Per 90 Minutes_xG,Per 90 Minutes_xAG,Per 90 Minutes_xG+xAG,Per 90 Minutes_npxG,Per 90 Minutes_npxG+xAG,Team,League_Previous_Season
187,Maxime Estève,fr FRA,DF,22.0,46.0,46,4055.0,45.1,1.0,0.0,...,0.02,0.02,0.02,0.02,0.00,0.02,0.02,0.02,Burnley,EFL Championship
188,James Trafford,eng ENG,GK,21.0,45.0,45,4050.0,45.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Burnley,EFL Championship
189,Josh Cullen,ie IRL,MF,28.0,44.0,43,3801.0,42.2,2.0,3.0,...,0.12,0.05,0.12,0.04,0.06,0.10,0.04,0.10,Burnley,EFL Championship
190,Jaidon Anthony,eng ENG,"FW,MF",24.0,43.0,42,3661.0,40.7,8.0,7.0,...,0.37,0.20,0.37,0.23,0.20,0.43,0.23,0.43,Burnley,EFL Championship
191,Connor Roberts,wls WAL,DF,28.0,41.0,40,3636.0,40.4,2.0,3.0,...,0.12,0.05,0.12,0.06,0.06,0.12,0.06,0.12,Burnley,EFL Championship
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586,Joe Anderson,eng ENG,DF,23.0,2.0,1,99.0,1.1,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Sunderland,EFL Championship
587,Ian Poveda,eng ENG,"FW,MF",24.0,6.0,0,86.0,1.0,0.0,0.0,...,0.00,0.00,0.00,0.06,0.17,0.23,0.06,0.23,Sunderland,EFL Championship
588,Nazariy Rusyn,ua UKR,FW,25.0,8.0,0,72.0,0.8,0.0,0.0,...,0.00,0.00,0.00,0.00,0.02,0.02,0.00,0.02,Sunderland,EFL Championship
589,Jenson Seelt,nl NED,DF,21.0,1.0,0,22.0,0.2,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Sunderland,EFL Championship


In [134]:
stat_df.rename(columns = {
    'League': 'League_Previous_Season'
}, inplace=True)

In [135]:
stat_df

,Player Name,Nation,Position,Age,Matches Played,Playing Time_Starts,Playing Time_Min,Playing Time_90s,Performance_Gls,Performance_Ast,...,Per 90 Minutes_G+A,Per 90 Minutes_G-PK,Per 90 Minutes_G+A-PK,Per 90 Minutes_xG,Per 90 Minutes_xAG,Per 90 Minutes_xG+xAG,Per 90 Minutes_npxG,Per 90 Minutes_npxG+xAG,Team,League_Previous_Season
0,David Raya,es ESP,GK,28.0,38.0,38,3420.0,38.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Arsenal,English Premier League
1,William Saliba,fr FRA,DF,23.0,35.0,35,3039.0,33.8,2.0,0.0,...,0.06,0.06,0.06,0.07,0.03,0.09,0.07,0.09,Arsenal,English Premier League
2,Declan Rice,eng ENG,MF,25.0,35.0,33,2825.0,31.4,4.0,7.0,...,0.35,0.13,0.35,0.11,0.21,0.32,0.11,0.32,Arsenal,English Premier League
3,Thomas Partey,gh GHA,"MF,DF",31.0,35.0,31,2797.0,31.1,4.0,2.0,...,0.19,0.13,0.19,0.07,0.07,0.14,0.07,0.14,Arsenal,English Premier League
4,Leandro Trossard,be BEL,FW,29.0,38.0,28,2546.0,28.3,8.0,7.0,...,0.53,0.28,0.53,0.25,0.22,0.47,0.25,0.47,Arsenal,English Premier League
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,Daniel Podence,pt POR,"MF,FW",28.0,2.0,0,48.0,0.5,0.0,0.0,...,0.00,0.00,0.00,0.07,0.00,0.07,0.07,0.07,Wolverhampton-Wanderers,English Premier League
697,Chiquinho,pt POR,MF,24.0,1.0,0,7.0,0.1,0.0,0.0,...,0.00,0.00,0.00,0.00,0.42,0.42,0.00,0.42,Wolverhampton-Wanderers,English Premier League
699,Tom King,wls WAL,GK,29.0,1.0,0,1.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Wolverhampton-Wanderers,English Premier League
700,Alfie Pond,eng ENG,DF,20.0,1.0,0,1.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Wolverhampton-Wanderers,English Premier League


In [143]:
stat_df[stat_df["Team"] == "Manchester-City"]

,Player Name,Nation,Position,Age,Matches Played,Playing Time_Starts,Playing Time_Min,Playing Time_90s,Performance_Gls,Performance_Ast,...,Per 90 Minutes_G+A,Per 90 Minutes_G-PK,Per 90 Minutes_G+A-PK,Per 90 Minutes_xG,Per 90 Minutes_xAG,Per 90 Minutes_xG+xAG,Per 90 Minutes_npxG,Per 90 Minutes_npxG+xAG,Team,League_Previous_Season
427,Joško Gvardiol,hr CRO,DF,22.0,37.0,36,3277.0,36.4,5.0,0.0,...,0.14,0.14,0.14,0.13,0.06,0.18,0.12,0.17,Manchester-City,English Premier League
428,Erling Haaland,no NOR,FW,24.0,31.0,31,2736.0,30.4,22.0,3.0,...,0.82,0.62,0.72,0.72,0.10,0.82,0.62,0.72,Manchester-City,English Premier League
429,Bernardo Silva,pt POR,"MF,FW",29.0,33.0,29,2670.0,29.7,4.0,4.0,...,0.27,0.13,0.27,0.13,0.17,0.30,0.13,0.30,Manchester-City,English Premier League
430,Ederson,br BRA,GK,30.0,26.0,26,2320.0,25.8,0.0,4.0,...,0.16,0.00,0.16,0.00,0.06,0.06,0.00,0.06,Manchester-City,English Premier League
431,Rúben Dias,pt POR,DF,27.0,27.0,25,2271.0,25.2,0.0,0.0,...,0.00,0.00,0.00,0.06,0.01,0.07,0.06,0.07,Manchester-City,English Premier League
432,İlkay Gündoğan,de GER,MF,33.0,33.0,25,2223.0,24.7,1.0,6.0,...,0.28,0.04,0.28,0.13,0.14,0.27,0.13,0.27,Manchester-City,English Premier League
433,Mateo Kovačić,hr CRO,MF,30.0,31.0,25,2198.0,24.4,6.0,2.0,...,0.33,0.25,0.33,0.08,0.15,0.23,0.08,0.23,Manchester-City,English Premier League
434,Manuel Akanji,ch SUI,DF,29.0,26.0,23,2014.0,22.4,0.0,0.0,...,0.00,0.00,0.00,0.07,0.02,0.09,0.07,0.09,Manchester-City,English Premier League
435,Rico Lewis,eng ENG,"DF,MF",19.0,28.0,21,1893.0,21.0,1.0,2.0,...,0.14,0.05,0.14,0.07,0.02,0.08,0.07,0.08,Manchester-City,English Premier League
436,Sávio,br BRA,"FW,MF",20.0,29.0,21,1766.0,19.6,1.0,8.0,...,0.46,0.05,0.46,0.26,0.35,0.61,0.26,0.61,Manchester-City,English Premier League


In [149]:
stat_df[stat_df["Performance_G+A"] >= 28]

,Player Name,Nation,Position,Age,Matches Played,Playing Time_Starts,Playing Time_Min,Playing Time_90s,Performance_Gls,Performance_Ast,...,Per 90 Minutes_G+A,Per 90 Minutes_G-PK,Per 90 Minutes_G+A-PK,Per 90 Minutes_xG,Per 90 Minutes_xAG,Per 90 Minutes_xG+xAG,Per 90 Minutes_npxG,Per 90 Minutes_npxG+xAG,Team,League_Previous_Season
398,Mohamed Salah,eg EGY,FW,32.0,38.0,38,3371.0,37.5,29.0,18.0,...,1.25,0.53,1.01,0.67,0.38,1.05,0.49,0.87,Liverpool,English Premier League
506,Alexander Isak,se SWE,FW,24.0,34.0,34,2756.0,30.6,23.0,6.0,...,0.95,0.62,0.82,0.66,0.14,0.81,0.56,0.70,Newcastle-United,English Premier League


In [150]:
stat_df.to_csv("epl_stats.csv")